# PromptLayer

<img src="https://promptlayer.com/logo.png"  height="300">


[PromptLayer](https://promptlayer.com) is a an observability platform for prompts and LLMs. In this guide we will go over how to setup the `PromptLayerCallbackHandler`. While PromptLayer does have LLMs that integrate directly with LangChain (eg [`PromptLayerOpenAI`](https://python.langchain.com/docs/modules/model_io/models/llms/integrations/promptlayer_openai)), this callback will be an easier and more feature rich way to integrate PromptLayer with any model on LangChain. 

This callback is also the recommended way to connect with PromptLayer when building Chains and Agents on LangChain.

## Installation and Setup

In [ ]:
!pip install promptlayer --upgrade

### Getting API Credentials

If you have not already create an account on [PromptLayer](https://www.promptlayer.com) and get an API key by clicking on the settings cog in the navbar
Set it as an environment variabled called `PROMPTLAYER_API_KEY`


### Usage

To get started with `PromptLayerCallbackHandler` is fairly simple, it takes two optional arguments:
1. `pl_tags` - an optional list of strings that will be tags tracked on PromptLayer
2. `pl_id_callback` - an optional function that will get a `promptlayer_request_id` as an argument. This id can be used with all of PromptLayers tracking features to track, metadata, scores, and prompt usage.

### Simple Example

In this simple example we use `PromptLayerCallbackHandler` with `ChatOpenAI`. We add a PromptLayer tag named `chatopenai`

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain.callbacks import PromptLayerCallbackHandler

chat_llm = ChatOpenAI(
    temperature=0,
    callbacks=[PromptLayerCallbackHandler(pl_tags=["chatopenai"])],
)
llm_results = chat_llm(
    [
        HumanMessage(content="What comes after 1,2,3 ?"),
        HumanMessage(content="Tell me another joke?"),
    ]
)
print(llm_results)


content="Sure, here's one:\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!" additional_kwargs={} example=False


### Full Featured Example

In this example we unlock more of the power of PromptLayer.

We are using the Prompt Registry and fetching the prompt called `example`.

We also define a `pl_id_callback` function that tracks a score, metadata and the prompt used. Read more about tracking on [our docs](docs.promptlayer.com).

In [4]:
from langchain.llms import OpenAI
from langchain.callbacks import PromptLayerCallbackHandler
import promptlayer

def pl_id_callback(promptlayer_request_id):
    print("prompt layer id ", promptlayer_request_id)
    promptlayer.track.score(
        request_id=promptlayer_request_id, score=100
    )  # score is an integer 0-100
    promptlayer.track.metadata(
        request_id=promptlayer_request_id, metadata={"foo": "bar"}
    )  # metadata is a dictionary of key value pairs that is tracked on PromptLayer
    promptlayer.track.prompt(
        request_id=promptlayer_request_id,
        prompt_name="example",
        prompt_input_variables={"product": "toasters"},
        version=1,
    )


openai_llm = OpenAI(
    model_name="text-davinci-002",
    callbacks=[PromptLayerCallbackHandler(pl_id_callback=pl_id_callback)],
)

example_prompt = promptlayer.prompts.get("example", version=1, langchain=True)
openai_llm(example_prompt.format(product="toasters"))

prompt layer id  6050929


'\nToasterCo.'

That is all it takes! After setup all your requests will show up on the PromptLayer dasahboard.
This callback also works with any LLM implemented on LangChain.